Pipeline to create the NEAT model.

There will be a collection of features to use in the analysis
- Total number of suspicious transaction based on the random forest model
- Total in / out going value of a transaction (this would be two different features)
- Total number of 0 seconds between transactions (could be evidence for structuring)
- Average number of daily transactions (another feature for structuring)
- need to do more research to find other features for the basis of model

Things I don't see working:
- Number of unique wallets money is sent to

need to look into saving the normalizer as well when the data is passed through because the machine has no idea what it's reading when passing in unnormalized data



In [1]:
# import packages 

import pandas as pd
from NEAT_get_transaction import get_transaction_history
import pickle
import os
import numpy as np
import sklearn

/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
api_key = '5M47SltcWwM6LG4DjNisTcSdNs175YsOUePfgmVKqYSAOFxPRaPaD2VRF4H4V4SH'
address = '0x098b716b8aaf21512996dc57eb0615e2383e2f96'

current_history = get_transaction_history(api_key, address)

0
1
2
3
4
Index(['index', 'to_address', 'from_address', 'value', 'gas',
       'block_timestamp', 'time_b/w_trans', 'day', 'month', 'hour',
       'type_of_trans'],
      dtype='object')


In [3]:
# get most updated random forest model
def get_current_rf(path):
    try:
        list_files = os.listdir(path)
        files = [file for file in list_files if file != '.DS_Store']
        files.sort()
        model = pickle.load(open(path + files[-1], 'rb'))
        return model
    except FileNotFoundError as fe:
        print(str(fe) + "\nReturning Object of Class None.")
        return None
    except pickle.UnpicklingError as pe:
        print("Error when unpickling file: " + str(pe) + ".\nReturning Object of Class None.")
        return None
    
def get_current_normalizer(path):
    try:
        list_files = os.listdir(path)
        files = [file for file in list_files if file != '.DS_Store']
        files.sort()
        model = pickle.load(open(path + files[-1], 'rb'))
        return model
    except FileNotFoundError as fe:
        print(str(fe) + "\nReturning Object of Class None.")
        return None
    except pickle.UnpicklingError as pe:
        print("Error when unpickling file: " + str(pe) + ".\nReturning Object of Class None.")
        return None


path = "/Users/matthewschultz/Big_Data_Lab/identity_review/iteration_rand_forest/"
model = get_current_rf(path)

scalar = get_current_normalizer("/Users/matthewschultz/Big_Data_Lab/identity_review/normalizer/")

In [4]:
to_be_normalized = scalar.transform(current_history[['value', 'time_b/w_trans']])

not_normalized = current_history[['day', 'hour', 'type_of_trans']].to_numpy()

combined = np.concatenate((to_be_normalized, not_normalized), axis = 1)


normalized = pd.DataFrame(combined)
normalized = normalized.rename(columns = {
    0: 'value',
    1: 'time_b/w_trans',
    2: 'day',
    3: 'hour',
    4: 'type_of_trans',
    })


predicted = []
for index, row in normalized.iterrows():
    predict = model.predict(row[['value', 'time_b/w_trans', 'day', 'hour','type_of_trans']].to_numpy().reshape(1, -1))
    predicted.append(predict)

normalized["malicious"] = predicted

num_of_mal_trans = (normalized.loc[normalized['malicious'] == 1])

This feature is done

In [5]:
value_out = current_history.loc[current_history['type_of_trans'] == 1]
value_out = sum(value_out['value'])
value_in = current_history.loc[current_history['type_of_trans'] == 0]
value_in = sum(value_in['value'])

print(value_out / value_in)


21.021000601482196


In [15]:
def get_avg_trans_per_day(df):
    months = [*set(df['month'])]
    days = [*set(df['day'])]
    day_by_day = [len(df.loc[df['block_timestamp'].str.contains('{}-{}'.format(month, day))]) for month in months for day in days]
    day_by_day = [val for val in day_by_day if val > 0]
    return sum(day_by_day) / len(day_by_day)
    
    

avg_trans = get_avg_trans_per_day(current_history)

num_of_zero = len(current_history.loc[current_history['time_b/w_trans'] == 0])